In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install dataprep

In [ ]:
from dataprep.eda import create_report
data = pd.read_csv('/kaggle/input/predict-test-scores-of-students/test_scores.csv')
# Convert categorical values to numbers
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
data.drop(['classroom', 'student_id'], axis = 1, inplace = True)
categorical_cols = data.select_dtypes(include='object').columns
# apply le on categorical feature columns

# data[categorical_cols] = data[categorical_cols].apply(lambda col: le.fit_transform(col))  
data_enc = pd.get_dummies(data, columns = categorical_cols)
report = create_report(data, title='test_scores Report')
report

In [ ]:
data_enc

In [ ]:
from tensorflow.keras.layers import BatchNormalization,Activation,MaxPool1D,Input,Dropout,Multiply,Reshape,Conv1D,AveragePooling1D,Flatten,Dense
import tensorflow_addons as tfa
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras import Model
def create_cnn( num_columns=36 , num_labels=1, dropout_rates=[0.2]):
    inp = Input(shape=(num_columns,))
    x = BatchNormalization()(inp)
    x = Dropout(dropout_rates[0])(x)
    x = WeightNormalization(Dense(4096))(x)
    x = Reshape((256, 16))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    x = WeightNormalization(Conv1D(filters = 16, kernel_size = 5, activation = 'swish', use_bias = False, padding = 'SAME'))(x)
    x = AveragePooling1D(pool_size = 2)(x)
    xs = x
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    x = WeightNormalization(Conv1D(filters = 16, kernel_size = 3, activation = 'swish', use_bias = True, padding = 'SAME'))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    x = WeightNormalization(Conv1D(filters = 16, kernel_size = 3, activation = 'swish', use_bias = True, padding = 'SAME'))(x)
    x = Multiply()([x, xs])
    x = MaxPool1D(pool_size = 4, strides = 2)(x)
    x = Flatten()(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    x = Dense(num_labels)(x)
    out = Activation('sigmoid')(x)
    model = Model(inputs=inp, outputs=out)
#     model.summary()
    return model

In [ ]:
# Split the data into X and y
X = data_enc.drop('posttest', axis = 1)
y = data_enc['posttest']
y = y.apply(lambda x:x/100)
X['pretest'] = X['pretest'].apply(lambda x:x/100)
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size = 0.3)

In [ ]:
model = create_cnn()
model.compile(optimizer='adam', 
                 loss='mae',
                 metrics=['accuracy'])

In [ ]:

model.fit(X_train,y_train,
            
            batch_size = 100,
            epochs = 150
           )
pred = model.predict(X_val_test)
mean_absolute_error(pred,y_val_test)

In [ ]:
pred = model.predict(X_val_test)
pred

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(pred,y_val_test)

In [ ]:
pd.DataFrame(list(zip(pred, y_val_test)))

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression().fit(X_train, y_train)

In [ ]:
clf.score(X_val_test,y_val_test)

In [ ]:
pred = clf.predict(X_val_test)
mean_absolute_error(pred,y_val_test)

In [ ]:
pd.DataFrame(list(zip(pred, y_val_test)))

In [ ]:
def create_nn( num_columns=36 , num_labels=1, dropout_rates=[0.2]):
    inp = Input(shape=(num_columns,))
    x = WeightNormalization(Dense(128))(inp)
    x = Dense(64,activation = 'sigmoid')(x)
    x = Dense(32,activation = 'sigmoid')(x)
    x = Dense(16,activation = 'sigmoid')(x)
    x = Dense(8,activation = 'sigmoid')(x)
    x = Dense(num_labels)(x)
    out = Activation('sigmoid')(x)
    model = Model(inputs=inp, outputs=out)
#     model.summary()
    return model

In [ ]:
model = create_nn()
model.compile(optimizer='adam', 
                 loss='mae',
                 metrics=['accuracy'])
model.fit(X_train,y_train,
            
            batch_size = 100,
            epochs = 150
           )


In [ ]:
pred = model.predict(X_val_test)
mean_absolute_error(pred,y_val_test)